In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
import seaborn as sns
from joblib import parallel, delayed
import math as m
import datetime
from seaborn import lmplot
from seaborn import distplot

In [ ]:
df_segmentados= pd.read_excel('/content/Segmentaciones_Nodos_Ejecutados.xlsx')
df_segmentados.head(5)

In [ ]:
df_segmentados.shape

In [ ]:
# # renombramos columnas
# df_segmentados.rename(columns={'Nodo':'NODO'}, inplace=True)
# df_segmentados.head(5)

In [ ]:
df_Nodosokla= pd.read_csv('/content/NODOS_INFO_OOKLA.csv',sep=';', header = 0)
df_Nodosokla.head(5)

In [ ]:
df_Nodosokla.shape

In [ ]:
df_Nodosokla['MES'] = [f'0{x}' if x <10 else str(x) for x in df_Nodosokla['MES']]
df_Nodosokla['FECHA'] = df_Nodosokla['ANIO'].astype('str') + '-' + df_Nodosokla['MES']

In [ ]:
df_Nodosokla = df_Nodosokla[['FECHA', 'NODO',	'ANIO',	'MES', 'OPERADOR', 'CANTIDAD',	'DOWNLOAD_KBPS',	'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT'	]]  

In [ ]:
df_Nodosokla

In [ ]:
df_Nodosokla.to_csv('okla2.csv') # Carga de datos

In [ ]:
# df_Nodosokla= pd.read_csv('/content/okla2.csv')
# df_Nodosokla.head(5)

In [ ]:
nodos=df_Nodosokla["NODO"].value_counts()
nodos_con_mas_registros=nodos[:30]
print(nodos_con_mas_registros)

In [ ]:
InfoByNodo=df_Nodosokla[['NODO','FECHA']].groupby('NODO').count()
InfoByNodo.describe()

In [ ]:
sampleDW=InfoByNodo[InfoByNodo['FECHA']==56].sort_values('FECHA',ascending=False).head(10)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
for ind, row in sampleDW.iterrows():   
    df_Nodosokla[df_Nodosokla['NODO']==ind].sort_values('FECHA',ascending=True).plot(x='FECHA',y='DOWNLOAD_KBPS',ax=ax, kind='line', label=ind)    

In [ ]:
# # para unir csv por nodos pero hay que unir el nombre Nodo=NODO=NOD0 en ambos csv
# clients1=pd.merge(df_Nodosokla, df_segmentados, on='NODO')
# clients1

In [ ]:
# essto me muestra que hay unna relacion lineal entre 'UPLOAD_KBPS', 'DOWNLOAD_KBPS'
lmplot('UPLOAD_KBPS', 'DOWNLOAD_KBPS', data=df_Nodosokla)

In [ ]:
from seaborn import lmplot
lmplot(x="DOWNLOAD_KBPS", y="UPLOAD_KBPS", hue="OPERADOR", data=df_Nodosokla)

In [ ]:
#  tendencia de datos para claro
lmplot(x="DOWNLOAD_KBPS", y="UPLOAD_KBPS", data=df_Nodosokla[df_Nodosokla['OPERADOR']=='CLARO'])

In [ ]:
df_Nodosokla.describe()

In [ ]:
sns.catplot(y="CANTIDAD", x="OPERADOR", palette="dark", data=df_Nodosokla)

In [ ]:
sns.catplot(y="DOWNLOAD_KBPS", x="OPERADOR", kind="violin", palette="dark", data=df_Nodosokla)

In [ ]:
# Operadorcon mas registros en okla
sns.countplot(x='OPERADOR', data=df_Nodosokla)

In [ ]:
# la latencia de red es la suma de retardos temporales dentro de una red. Un retardo es producido por la demora en la propagación y transmisión de paquetes dentro de la red. 
from seaborn import distplot
distplot(df_Nodosokla.LATENCY)

In [ ]:
operadores=df_Nodosokla["OPERADOR"].value_counts(ascending=False)#.tolist()
print(operadores)
print('suma de operadores registrados en okla',sum(operadores))

In [ ]:
veldescarga=df_Nodosokla["DOWNLOAD_KBPS"].value_counts(ascending=False)#.tolist()
veldescarga=veldescarga[:10]
print(veldescarga)

In [ ]:
velsubida=df_Nodosokla["UPLOAD_KBPS"].value_counts(ascending=False)#.tolist()
velsubida=velsubida[:10]
print(velsubida)

In [ ]:
df_Nodosokla.groupby(by='OPERADOR')['DOWNLOAD_KBPS','UPLOAD_KBPS','LATENCY',	'FLAG_LAT'].max() # nos agrupa por operador y por cada tipo de funcion nos aplica la funcion maxima aplicado a cada operador

In [ ]:
df_Nodosokla.groupby(by='OPERADOR')['DOWNLOAD_KBPS','UPLOAD_KBPS','LATENCY',	'FLAG_LAT'].mean()

In [ ]:
df_Nodosokla.groupby(by='OPERADOR')['DOWNLOAD_KBPS','UPLOAD_KBPS','LATENCY',	'FLAG_LAT'].describe().T

In [ ]:
gb_df= pd.DataFrame(df_Nodosokla.groupby(['OPERADOR','UPLOAD_KBPS'],as_index=False)['LATENCY'].mean())
gb_df

In [ ]:
from seaborn import boxplot
boxplot(df_Nodosokla.DOWNLOAD_KBPS, orient="v")

In [ ]:
# sns.jointplot(x='FECHA',y='LATENCY',data=df_Nodosokla, kind='scatter',height=10, ratio=3,)

In [ ]:
sns.jointplot(x='UPLOAD_KBPS',y='LATENCY',data=df_Nodosokla)

In [ ]:
sns.jointplot(x='DOWNLOAD_KBPS',y='LATENCY',data=df_Nodosokla)

In [ ]:
from seaborn import lmplot
lmplot('DOWNLOAD_KBPS', 'LATENCY', data=df_Nodosokla)

In [ ]:
# distribución de la variable DOWNLOAD_KBPS
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6, 6))
sns.distplot(
    df_Nodosokla.DOWNLOAD_KBPS,
    hist    = False,
    rug     = True,
    color   = "blue",
    kde_kws = {'shade': True, 'linewidth': 1},
    ax      = axes[0]
)
axes[0].set_title("Distribución original", fontsize = 'medium')
axes[0].set_xlabel('Download', fontsize='small') 
axes[0].tick_params(labelsize = 6)

sns.distplot(
    np.sqrt(df_Nodosokla.DOWNLOAD_KBPS),
    hist    = False,
    rug     = True,
    color   = "blue",
    kde_kws = {'shade': True, 'linewidth': 1},
    ax      = axes[1]
)
axes[1].set_title("Transformación raíz cuadrada", fontsize = 'medium')
axes[1].set_xlabel('sqrt(Download)', fontsize='small') 
axes[1].tick_params(labelsize = 6)

sns.distplot(
    np.log(df_Nodosokla.DOWNLOAD_KBPS),
    hist    = False,
    rug     = True,
    color   = "blue",
    kde_kws = {'shade': True, 'linewidth': 1},
    ax      = axes[2]
)
axes[2].set_title("Transformación logarítmica", fontsize = 'medium')
axes[2].set_xlabel('log(Download)', fontsize='small') 
axes[2].tick_params(labelsize = 6)

fig.tight_layout()

In [ ]:
# distribución de la variable UPLOAD_KBPS
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6, 6))
sns.distplot(
    df_Nodosokla.UPLOAD_KBPS,
    hist    = False,
    rug     = True,
    color   = "blue",
    kde_kws = {'shade': True, 'linewidth': 1},
    ax      = axes[0]
)
axes[0].set_title("Distribución original", fontsize = 'medium')
axes[0].set_xlabel('Upload', fontsize='small') 
axes[0].tick_params(labelsize = 6)

sns.distplot(
    np.sqrt(df_Nodosokla.UPLOAD_KBPS),
    hist    = False,
    rug     = True,
    color   = "blue",
    kde_kws = {'shade': True, 'linewidth': 1},
    ax      = axes[1]
)
axes[1].set_title("Transformación raíz cuadrada", fontsize = 'medium')
axes[1].set_xlabel('sqrt(Upload)', fontsize='small') 
axes[1].tick_params(labelsize = 6)

sns.distplot(
    np.log(df_Nodosokla.UPLOAD_KBPS),
    hist    = False,
    rug     = True,
    color   = "blue",
    kde_kws = {'shade': True, 'linewidth': 1},
    ax      = axes[2]
)
axes[2].set_title("Transformación logarítmica", fontsize = 'medium')
axes[2].set_xlabel('log(Upload)', fontsize='small') 
axes[2].tick_params(labelsize = 6)

fig.tight_layout()

In [ ]:
x = df_Nodosokla['CANTIDAD'].values # returns # CANTIDAD	DOWNLOAD_KBPS	UPLOAD_KBPS	LATENCY	FLAG_LAT
x_str='Real returns'  # label e,g ric
x_size= len(x)  #  retorna el tamaño df_Nodosokla['CANTIDAD'].shape
print(x_size)
print(df_Nodosokla.info())

In [ ]:
# round_digits = 4
from scipy.stats import skew, kurtosis, chi2

x_mean = np.mean(x) # para calcular la  media
x_std =np.std(x) # para la desviacion estandar o volatilidad si se habla de mercados
x_skew = skew(x) # para calcular la skweness o simetria
x_kurt = kurtosis(x) # curtosis en exceso, para calcular la kurtosis y ver que tantas colas largas tiene
x_sharpe = x_mean/x_std # esto es el rendimiento en unidades de riesgo
x_var_95 = np.percentile(x,95) # valor en riesgo esto nos dice que solamente el 5% de los valores son mayores, el valor arrojado en este caso mayores a 1.64
x_cvar_95 = np.mean(x[x <= x_var_95]) # valor en riesgo condicional, esto nos dice, que cuando pierdo que tanto pierdo, esto es el promedio de todos los valores que estan a la isquierda osea que se encuentran en el -5% en este caso
x_sharpe = x_mean / x_std * np.sqrt(252) # indica el rendimiento esperado por unidad de riesgo 252 eselnumero de dias anuales trabajados
x_corrcoef = np.corrcoef(x) # coeficiente de correlacion
x_var = np.var(x) # covarianza
# Test de jarque-bera
jb = x_size/6*(x_skew**2 + 1/4*x_kurt**2) # jarque bera
# p-value
p_value = 1 - chi2.cdf(jb, df=2) # se deben fijar df en 2 para la t-student, df= grados de liberdad 
is_normal = (p_value > 0.05) # con esto necesitamos que equivalently jb < 6

In [ ]:
print('media               =', x_mean)
print('desviacion estandar =', x_std) # indica qué tan dispersos están los datos con respecto a la media para la desviacion estandar o volatilidad si se habla de mercado
print('skweness            =', x_skew) # skweness es una variable aleatoria
print('kurtosis            =', x_kurt) # curtosis en exceso, determina el grado de concentración que presentan los valores de una variable alrededor de la zona central de la distribución de frecuencias
print('x_sharpe            =', x_sharpe)
print('valor en riesgo     =', x_var_95) # Valor en riesgo esto nos dice que solamente el 5% de los valores son mayores, el valor arrojado en este caso mayores
print('riesgo condicional  =', x_cvar_95)
print('x_sharpe            =', x_sharpe)
print('Coefic correlacion  =', x_corrcoef)
print('varianza            =', x_var)
print('jarque bera         =', jb) # si se tiene un jarque bera > 6 la distribucion no es normal
print('p_value             =', p_value) # p < 0,05 significa que la hipótesis nula es falsa y una p > 0,05 que la hipótesis nula es verdadera: # esto me da la probabilidad de que suponiendo que tenga una distribicion chi2, cual seria la probabilidad # de tener puntos que caigan a la izquierda de dicho valor
print('is_normal           =', is_normal) 

In [ ]:
# para ver el grado de correlación entre las variables disponibles.
def tidy_corr_matrix(corr_mat):
    '''
    Función para convertir una matrix de correlación de pandas en formato tidy
    '''
    corr_mat = corr_mat.stack().reset_index()
    corr_mat.columns = ['variable_1','variable_2','r']
    corr_mat = corr_mat.loc[corr_mat['variable_1'] != corr_mat['variable_2'], :]
    corr_mat['abs_r'] = np.abs(corr_mat['r'])
    corr_mat = corr_mat.sort_values('abs_r', ascending=False)
    
    return(corr_mat)



corr_matrix = df_Nodosokla.select_dtypes(include=['float64', 'int']).corr(method='pearson')
tidy_corr_matrix(corr_matrix).head(10)

In [ ]:
# Heatmap matriz de correlaciones
# ==============================================================================
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(22, 12))

sns.heatmap(
    corr_matrix,
    annot     = True,
    cbar      = False,
    annot_kws = {"size": 1},
    vmin      = -1,
    vmax      = 1,
    center    = 0,
    cmap      = sns.diverging_palette(20, 220, n=200),
    square    = True,
    ax        = ax
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation = 45,
    horizontalalignment = 'right',
)

ax.tick_params(labelsize = 8)

In [ ]:
# # modelo xgboost
# import xgboost as xgb
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedKFold
# from xgboost import XGBRegressor
# from xgboost import XGBRFClassifier, XGBClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LogisticRegression # clasificador linear
# from sklearn.metrics import confusion_matrix # es una manera para resumir los datos 

# from sklearn.metrics import mean_squared_error as mse
# from sklearn.datasets import load_boston
# from sklearn.model_selection import train_test_split
# from yellowbrick.regressor import residuals_plot
# from yellowbrick.regressor import prediction_error
# from sklearn.metrics import r2_score
# from sklearn.metrics import accuracy_score
# from scipy.stats import chi2_contingency
# from sklearn.covariance import EllipticEnvelope

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# import statsmodels.formula.api as smf
# import statsmodels.api as sm
# from sklearn.metrics  import r2_score, mean_squared_error # metrica para medir


# cols_entrenamiento= ['CANTIDAD', 'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT']


# X = df_Nodosokla[cols_entrenamiento] #.values.reshape(-1, 1)
# y= df_Nodosokla['DOWNLOAD_KBPS'] # score es una columna, esto quiere decir que de acuerdo a x determinamos y

# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=22) 


# xgb_reg = xgb.XGBRegressor(max_depth=3, n_estimators=100, n_jobs=2,
#                            objective='reg:squarederror', 
#                            booster='gbtree',
#                            random_state=42, learning_rate=0.05)

# # Train the model with train data sets
# xgb_reg.fit(X_train, y_train)


# # x_p=dfB[cols_prediccion]

# y_pred = xgb_reg.predict(X_test) # Predictions
# y_true = y_test # True values

# # y_pred = xgb_reg.predict(X)
# # y_pred_test = xgb_reg.predict(x_test)

# MSE = mse(y_true, y_pred)
# RMSE = np.sqrt(MSE)

# R_squared = r2_score(y_test, y_pred)
# print(y_train.shape)
# print(y_test.shape)
# print(y_pred.shape)
# print("\nRMSE: ", np.round(RMSE, 2))
# print('y_pred',y_pred.shape)
# print("R-Squared: ", np.round(R_squared, 2))
# print('exactitud ',accuracy_score)

# predicions=pd.DataFrame(y_pred,columns=['DOWNLOAD_KBPS'])
# print(predicions)
# print(predicions.shape)
# # # predicions.to_csv('valor_total_avaluo.csv')


In [ ]:
# import xgboost as xgb
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedKFold
# from xgboost import XGBRegressor
# from xgboost import XGBRFClassifier, XGBClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LogisticRegression # clasificador linear
# from sklearn.metrics import confusion_matrix # es una manera para resumir los datos 
# from sklearn import tree
# from sklearn.metrics import mean_squared_error as mse
# from sklearn.datasets import load_boston
# from sklearn.model_selection import train_test_split
# from yellowbrick.regressor import residuals_plot
# from yellowbrick.regressor import prediction_error
# from sklearn.metrics import r2_score
# from sklearn.metrics import accuracy_score
# from scipy.stats import chi2_contingency
# from sklearn.covariance import EllipticEnvelope
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.datasets import make_classification

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# import statsmodels.formula.api as smf
# import statsmodels.api as sm
# from sklearn.metrics  import r2_score, mean_squared_error # metrica para medir
# from sklearn.metrics import mean_squared_error as mse


# clf = DecisionTreeRegressor(criterion='mse',max_depth=100, random_state=0)

# x = df_Nodosokla[['CANTIDAD', 'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT']]
# # x_test = df_Nodosokla[['CANTIDAD', 'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT']]
# y = df_Nodosokla['DOWNLOAD_KBPS'] 
# # y_test = df_Nodosokla['DOWNLOAD_KBPS'] 

# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=22) 

# RMSE = np.sqrt(MSE)

# clf = clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)


# # print(' R_cuadrado ', r2_score(y_test, y_pred))
# print("\nRMSE: ", np.round(RMSE, 2))
# print("R-Squared: ", np.round(R_squared, 2))

In [ ]:
# # modelo de Decision Tree Regressor
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LogisticRegression # clasificador linear
# from sklearn.datasets import make_classification
# from sklearn.preprocessing import StandardScaler # StandardScaler sive para normalizar nuestros datos osea  para que esten en la misma escala
# from sklearn.model_selection import train_test_split # para poder partir nuestros datos
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import r2_score


# cols_entrenamiento= ['CANTIDAD', 'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT']

# X = df_Nodosokla[cols_entrenamiento]
# y= df_Nodosokla['DOWNLOAD_KBPS']
# # En nuestros features tendremos definidos 155 registros, uno por cada pais, 7 columnas 1 por cada pais 
# print('X.shape',X.shape)
# # Y 155 para nuestra columna para nuestro target 
# print('y.shape',y.shape)
# # tendremos 25% de nuestros datos para test y 75% para train 
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=22) 

# dt_reg=DecisionTreeRegressor(criterion='mse',max_depth=100, random_state=0)


# dt_reg.fit(X_train,y_train)
# y_pred = dt_reg.predict(X_test)
# accuracy_score = dt_reg.score(X_test,y_test)

# # y_pred = dt_reg.predict(X_test)


# MSE = mse(y_true, y_pred)
# RMSE = np.sqrt(MSE)

# R_squared = r2_score(y_test, y_pred)
# print('y_train',y_train.shape)
# print('y_test',y_test.shape)
# print("\nRMSE: ", np.round(RMSE, 2))
# print('y_pred',y_pred.shape)

# print("R-Squared: ", np.round(R_squared, 2)) # RMSE indican un mejor ajuste.
# print('exactitud ',accuracy_score)


In [ ]:
# # modelo de regrecion linear
# from sklearn import datasets, linear_model # este ultimo para ajustar el modelo
# from sklearn.metrics import r2_score
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import Lasso
# from sklearn.linear_model import Ridge

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib

# # La función sample() devuelve de una lista de elementos un determinado número de elementos diferentes elegidos al azar.
# np.random.seed(0) # semilla


# cols_entrenamiento= ['CANTIDAD', 'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT']

# X = df_Nodosokla[cols_entrenamiento] #.values.reshape(-1, 1)
# y= df_Nodosokla['DOWNLOAD_KBPS'] # score es una columna, esto quiere decir que de acuerdo a x determinamos y

# # tendremos 25% de nuestros datos para test y 75% para train 
# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=22, shuffle= True) 


# # Creación del modelo
# # ==============================================================================
# modelo = LinearRegression()
# modelo.fit(X = X_train, y = y_train)

# print("Intercept:", modelo.intercept_)
# print("Coeficiente:", list(zip(X.columns, modelo.coef_.flatten(), )))
# print("Coeficiente de determinación R^2:", modelo.score(X, y))

# # Error de test del modelo 
# # ==============================================================================
# predicciones = modelo.predict(X = X_test)
# print(predicciones[0:3,])

# rmse = mean_squared_error(
#         y_true  = y_test,
#         y_pred  = predicciones,
#         squared = False
#        )
# print("")
# print(f"El error (rmse) de test es: {rmse}")

In [ ]:
# Modelos unificados
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from xgboost import XGBRFClassifier, XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression # clasificador linear
from sklearn.metrics import confusion_matrix # es una manera para resumir los datos 

from sklearn.metrics import mean_squared_error as mse
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from yellowbrick.regressor import residuals_plot
from yellowbrick.regressor import prediction_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from scipy.stats import chi2_contingency
from sklearn.covariance import EllipticEnvelope

from sklearn import datasets, linear_model # este ultimo para ajustar el modelo

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler # StandardScaler sive para normalizar nuestros datos osea  para que esten en la misma escala


from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics  import r2_score, mean_squared_error # metrica para medir


if __name__ == "__main__":
  # La función sample() devuelve de una lista de elementos un determinado número de elementos diferentes elegidos al azar.
  np.random.seed(0) # semilla

  # Importamos el dataset del 2017 
  # df_Nodosokla= pd.read_csv('/content/NODOS_INFO_OOKLA.csv',sep=';', header = 0)
  # Mostramos el reporte estadistico
  print('datos ',df_Nodosokla.describe())
  df_Nodosokla= df_Nodosokla.sample(frac=1) # reoredenamos los datos


  cols_entrenamiento= ['CANTIDAD', 'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT']

  X = df_Nodosokla[cols_entrenamiento] #.values.reshape(-1, 1)
  y= df_Nodosokla['DOWNLOAD_KBPS'] # score es una columna, esto quiere decir que de acuerdo a x determinamos y

  # tendremos 25% de nuestros datos para test y 75% para train 
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=22, shuffle= True) 

  estimadores = {
      'modelxgboot' : xgb.XGBRegressor(max_depth=3, n_estimators=100, n_jobs=2, objective='reg:squarederror', booster='gbtree', random_state=42, learning_rate=0.05),
      'modelDTree' : DecisionTreeRegressor(criterion='mse',max_depth=100, random_state=0),
      'modelLinear' : LinearRegression(),
      'modelLasso' : Lasso(alpha=0.2) , 
      'modelRidge' : Ridge(alpha=1)  
    }
  for name, estimator in estimadores.items():
      modelo=estimator.fit(X_train, y_train) # ajustamos los datos de entrenamiento
      predictions = estimator.predict(X_test)

      MSE = mse(y_test, predictions) # el MSE es el error cuadrático medio de un estimador mide el promedio de los errores al cuadrado, es decir, la diferencia entre el estimador y lo que se estima. El ECM es una función de riesgo, correspondiente al valor esperado de la pérdida del error al cuadrado o pérdida cuadrática.
      RMSE = np.sqrt(MSE)
      R_squared = r2_score(y_test, predictions)

      print("="*64) # estos nos imprimira 64 veces = antes de sacar nuestros datos
      print(name)
      print("MSE: ", mean_squared_error(y_test, predictions)) # imprimimos nuestro error pasandole
      print("\nRMSE: ", np.round(RMSE, 2))
      print("R-Squared: ", np.round(R_squared, 2))
      print('exactitud ',accuracy_score)

        # nuestros datos de prueba y nuestras preciones
        # nuestros datos de prueba y nuestras preciones
  for name, estimator in estimadores.items():
      print("="*32)
      print(name)    
      print(modelo.coef_)

# R_squared = r2_score(y_test, y_pred)
# print(y_train.shape)
# print(y_test.shape)
# print(y_pred.shape)

# print('y_pred',y_pred.shape)
# print("R-Squared: ", np.round(R_squared, 2))
# print('exactitud ',accuracy_score)


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_pred.shape)

In [ ]:
# tratando de comparar variables
# df_Nodosokla['NODO']=df_Nodosokla['NODO'].str.upper()
# df_segmentados['Nodo']=df_segmentados['Nodo'].str.upper()

# df=df_Nodosokla['Check']=df_segmentados.Nodo.isin(df_Nodosokla)
# print(df_Nodosokla['Check'].value_counts())

# print(df_Nodosokla.loc[df_Nodosokla.Check==True,'NODO'])

# df_Nodosokla

In [ ]:
# df_shot=df_Nodosokla[df_Nodosokla["Check"]=='True']
# df_shot.head(3)

In [ ]:
# # renombramos columnas
# df_segmentados.rename(columns={'Nodo':'NODO'}, inplace=True)
# df_segmentados.head(5)

In [ ]:
# # para unir csv por nodos pero hay que unir el nombre Nodo=NODO=NOD0 en ambos csv
# clients1=pd.merge(df_Nodosokla, df_segmentados, on='NODO')
# clients1

In [ ]:
# df_Nodosokla.merge(df_segmentados, on="NODO", how="left")

In [ ]:
# ccn=pd.concat([df_Nodosokla, df_segmentados], axis=1)
# ccn

In [ ]:
# outerjoin=df_Nodosokla[df_Nodosokla['NODO']].merge(df_segmentados[df_segmentados['Nodo']], how ='outer', indicator=True)
# print(outerjoin)

In [ ]:
# # https://www.analyticslane.com/2018/09/10/unir-y-combinar-dataframes-con-pandas-en-python/
# Newdf = pd.concat([df_Nodosokla, df_segmentados])
# Newdf

In [ ]:
# df_sho=Newdf[Newdf["NODO"]=='ILA']
# df_sho.head(3)

In [ ]:
# CANTIDAD', 'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT' 'DOWNLOAD_KBPS'
df_Nodosokla.drop(['DOWNLOAD_KBPS', 'UPLOAD_KBPS'],1).hist()
plt.show()

In [ ]:
# https://www.aprendemachinelearning.com/regresion-lineal-en-espanol-con-python/

filtered_data = df_Nodosokla[(df_Nodosokla['DOWNLOAD_KBPS'] <= 100) & (df_Nodosokla['UPLOAD_KBPS'] >= 5)]
 
colores=['orange','blue']
tamanios=[30,60]
 
f1 = filtered_data['DOWNLOAD_KBPS'].values
f2 = filtered_data['UPLOAD_KBPS'].values
 
# Vamos a pintar en colores los puntos por debajo y por encima de la media de Cantidad de Palabras
asignar=[]
for index, row in filtered_data.iterrows():
    if(row['DOWNLOAD_KBPS']>100):
        asignar.append(colores[0])
    else:
        asignar.append(colores[1])
    
plt.scatter(f1, f2, c=asignar, s=tamanios[0])
plt.show()

In [ ]:
# https://relopezbriega.github.io/blog/2016/09/26/series-de-tiempo-con-python/
df_Nodosokla.set_index('FECHA',inplace=True)

In [ ]:
# df_Nodosokla= df_Nodosokla.reset_index()

In [ ]:
df_Nodosokla

In [ ]:
# 'CANTIDAD', 'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT'
df_Nodosokla['CANTIDAD'].plot(linewidth=0.1)

In [ ]:
# graficando Adj Close
plot = df_Nodosokla['CANTIDAD'].plot(figsize=(10, 8))

In [ ]:
# Aplicando el filtro Hodrick-Prescott para separar en tendencia y 
# componente ciclico.
wft_ciclo, wft_tend = sm.tsa.filters.hpfilter(df_Nodosokla['DOWNLOAD_KBPS'])
df_Nodosokla['tend'] = wft_tend

# graficando la variacion  precio real con la tendencia.
df_Nodosokla[['DOWNLOAD_KBPS', 'tend']].plot(figsize=(10, 8), fontsize=12);
legend = plt.legend()
legend.prop.set_size(14);

In [ ]:
# calculando el porcentaje de variación de la semana
variacion_diaria = df_Nodosokla['DOWNLOAD_KBPS'] / df_Nodosokla['DOWNLOAD_KBPS'].shift(1) - 1
df_Nodosokla['var_semanal'] = variacion_diaria
df_Nodosokla['var_semanal'][:5]

In [ ]:
plot = df_Nodosokla['var_semanal'].plot(figsize=(10, 8))

In [ ]:
df_Nodosokla=df_Nodosokla[[	'NODO', 'ANIO',	'MES','OPERADOR','CANTIDAD', 'DOWNLOAD_KBPS', 'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT', 'tend',	'var_semanal'	]].dropna()

In [ ]:
df_Nodosokla

In [ ]:
# # modelo ARIMA sobre variación diaria
# modelo = sm.tsa.ARIMA(df_Nodosokla['var_semanal'].iloc[1:], order=(1, 0, 0))  
# resultados = modelo.fit(disp=-1)
# df_Nodosokla['pronostico'] = resultados.fittedvalues  
# plot = df_Nodosokla[['var_semanal', 'pronostico']].plot(figsize=(10, 8)) 

In [ ]:
# para ver el Número de datos ausentes por variable
df_Nodosokla.isnull().sum().sort_values(ascending=False,)[:21]

In [ ]:
df_Nodosokla

In [ ]:
df_Nodosokla= df_Nodosokla.sample(frac=1) # reoredenamos los datos

In [ ]:
# confusion_matrix=confusion_matrix(y_test, y_pred)
# print(confusion_matrix)

In [ ]:
# from sklearn import tree

# clf = tree.DecisionTreeClassifier() # create objet

# x = df_Nodosokla[['CANTIDAD', 'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT', 'DOWNLOAD_KBPS']]
# x_test = df_Nodosokla[['CANTIDAD', 'UPLOAD_KBPS',	'LATENCY',	'FLAG_LAT', 'DOWNLOAD_KBPS']]
# y = df_Nodosokla['NODO'] 

# clf = clf.fit(x,y) # Train objet
# y_pred = clf.predict(x)
# y_pred_test = clf.predict(x_test)

# np.mean(y==y_pred) # comparamos que % se predice correctamente
# # esta nos dice que el modelo predice muy bien la primera particion de los datos
# np.mean(y_pred == y_pred_test) # este modelo  predice la particion del dftest
# # y nos indica el % correctopredecido por este modelo
# print(' R_cuadrado ', r2_score(y, y_pred))
# print(' R_cuadrado ', r2_score(y_test, y_pred_test))